In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader
from datasets import Dataset
import numpy as np
from bd import nova_conexao
from psycopg2.errors import ProgrammingError
import pandas as pd
import re
import torch
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm
from utils import ndcg
import evaluate

# Obtendo os dados

In [ ]:
 # Obtendo os dados e colocando-as em um dataframe
def obter_dados_bd():
    sql_select_dados = "select eua.id, eua.curriculo_texto, eva.id, eva.texto, na.nota \
    from nota_analise na inner join emprega_usuario_analise eua on na.usuario_id = eua.id \
    inner join emprega_vaga_analise eva on na.vaga_id = eva.id \
    order by na.vaga_id"
    curriculos = []
    vagas = []
    notas = []
    with nova_conexao() as conexao:
        try:
            cursor = conexao.cursor()
            cursor.execute(sql_select_dados)
            resultado = cursor.fetchall()
        except ProgrammingError as e:
            print(f'Erro: {e.msg}')
        else:
            for row in resultado:
                curriculos.append(row[1])
                vagas.append(row[3])
                notas.append(row[4])
            conexao.commit()

    df_dados = pd.DataFrame(
        {
            'curriculos' : curriculos,
            'vagas' : vagas,
            'notas' : notas
        })
    df_dados.to_csv('dados.csv', index=False, encoding='utf-8')
    return df_dados


# Lendo e salvando para CSV
def obter_dados_csv():
    df_dados = pd.read_csv('dados.csv')
    df_dados.to_csv('dados.csv', index=False, encoding='utf-8')
    return df_dados

In [ ]:
df_dados = obter_dados_csv()
df_dados

# Removendo os dígitos

In [ ]:
df_dados["curriculos"] = df_dados["curriculos"].apply(lambda x: re.sub('\d+', '', x))
df_dados["vagas"] = df_dados["vagas"].apply(lambda x: re.sub('\d+', '', x))
df_dados["notas"] = df_dados["notas"].apply(lambda x: x-1) # Ajustando o target porque o torch utiliza as classes começando do 0 [0, num_labels-1]
df_dados

# Configurando o Transformer e o Tokenizer

In [ ]:
# Transforma em um dataset do HuggingFace para usar a sua função map
dados = Dataset.from_pandas(df_dados)
dados

In [ ]:
def tokenize_function(inputs):
    return tokenizer(
        inputs['curriculos'], inputs['vagas'], truncation=True, return_tensors='pt', padding='max_length', max_length=128
        )


tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
dados_tokenizados = dados.map(tokenize_function, batched=True)

model = AutoModelForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=5, hidden_dropout_prob = 0.1, output_hidden_states=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:
dados_tokenizados = dados_tokenizados.remove_columns(['curriculos', 'vagas'])
dados_tokenizados = dados_tokenizados.rename_column('notas', 'labels')
dados_tokenizados.set_format("torch")

In [ ]:
dados_tokenizados

# Preparando o treinamento

In [ ]:
def evaluate_model(eval_dataloader, desc='validação'):

    metric_acc = evaluate.load("accuracy")
    metric_f1_micro = evaluate.load("f1")
    metric_f1_macro = evaluate.load("f1")
    metric_f1_weighted = evaluate.load("f1")
    metric_roc_auc = evaluate.load("roc_auc", "multiclass")
    progress_bar2 = tqdm(range(len(eval_dataloader)), desc=f'Validação no conjunto de {desc}')
    model.eval()

    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        predictions = torch.argmax(outputs.logits, dim=-1)
        softmax_roc_auc = torch.softmax(outputs.logits, dim=-1) # Utilizado para o ROC_AUC que não usa os logits, mas a saída do softmax (a prob de cada classe, de modo que a soma de todas as prob dê 1)
        metric_acc.add_batch(predictions=predictions, references=batch["labels"])
        metric_f1_micro.add_batch(predictions=predictions, references=batch["labels"])
        metric_f1_macro.add_batch(predictions=predictions, references=batch["labels"])
        metric_f1_weighted.add_batch(predictions=predictions, references=batch["labels"])
        metric_roc_auc.add_batch(prediction_scores=softmax_roc_auc, references=batch["labels"])
        
        progress_bar2.update(1)
        
    acc = metric_acc.compute()
    f1_micro = metric_f1_micro.compute(average="micro")
    f1_macro = metric_f1_macro.compute(average="macro")
    f1_weighted = metric_f1_weighted.compute(average="weighted")
    roc_auc =  metric_roc_auc.compute(multi_class='ovo') #'ovo' é melhor para datasets desbalanceados
    
    return acc, f1_micro, f1_macro, f1_weighted, roc_auc

In [ ]:
dados_tokenizados=dados_tokenizados.train_test_split(0.2, seed=42)
test_dataloader = DataLoader(dados_tokenizados['test'], batch_size=8)

# A partir do treino, obtém os dados de validação
dados_tokenizados=dados_tokenizados['train'].train_test_split(0.2, seed=42)
train_dataloader = DataLoader(dados_tokenizados['train'], shuffle=True, batch_size=8)
val_dataloader = DataLoader(dados_tokenizados['test'], batch_size=8)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
optimizer = AdamW(model.parameters(), lr=5e-5)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

dados_tokenizados

# Treinamento

In [ ]:
progress_bar = tqdm(range(num_training_steps), desc='Treino')
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    train_acc, train_f1_micro, train_f1_macro, train_f1_weighted, train_roc_auc = evaluate_model(train_dataloader)
    val_acc, val_f1_micro, val_f1_macro, val_f1_weighted, val_roc_auc = evaluate_model(val_dataloader)
    print(f'Resultados com o conjunto de treino na época {epoch}')
    print(f'Acurácia: {train_acc}')
    print(f'Micro-F1: {train_f1_micro}')
    print(f'Macro-F1: {train_f1_macro}')
    print(f'Weighted-F1: {train_f1_weighted}')
    print(f'ROC-AUC: {train_roc_auc}')

    print(f'Resultados com o conjunto de validação na época {epoch}')
    print(f'Acurácia: {val_acc}')
    print(f'Micro-F1: {val_f1_micro}')
    print(f'Macro-F1: {val_f1_macro}')
    print(f'Weighted-F1: {val_f1_weighted}')
    print(f'ROC-AUC: {val_roc_auc}')

In [ ]:
    # a melhor métrica será macro f1 porque o dataset está desbalanceado
    # para a métrica f1_weighted, devemos selecionar melhor os dados de validação para colocar mais amostras onde se tem nota 5